In [1]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/conditional-tvaes.csv')

In [3]:
drop_columns = [
    'ISC.resultadosCovid.numDosis',
    'ISC.resultadosCovid.numVecesCovid',
    'ISC.resultadosCovid.numVecesCovid_mode',
    'ISC.resultadosCovid.pasadoCovid',
    'ISC.resultadosCovid.sentimientoAislado',
    'ISC.resultadosCovid.sentimientoAislado_mediaFiltro',
    'ISC.resultadosCovid.sentimientoAislado_mode',
    'ISC.resultadosCovid.sintomaCansancio',
    'ISC.resultadosCovid.sintomaDiarrea',
    'ISC.resultadosCovid.sintomaDifRespirar',
    'ISC.resultadosCovid.sintomaDolCabeza',
    'ISC.resultadosCovid.sintomaDolGarganta',
    'ISC.resultadosCovid.sintomaDolMuscular',
    'ISC.resultadosCovid.sintomaEscalofrios',
    'ISC.resultadosCovid.sintomaFiebre',
    'ISC.resultadosCovid.sintomaOtro',
    'ISC.resultadosCovid.sintomaOtroString',
    'ISC.resultadosCovid.sintomaPerGusto',
    'ISC.resultadosCovid.sintomaPerOlfato',
    'ISC.resultadosCovid.vacunado',
    'ISC.resultadosObservacion.añosSinFumar',
    'ISC.resultadosObservacion.añosSinFumar_Rangos',
    'ISC.resultadosObservacion.consumoAlcohol_UBEsemanal',
    'ISC.resultadosObservacion.consumoAlcohol_UBEsemanal_knn',
    'ISC.resultadosObservacion.consumoAlcohol_UBEsemanal_mode&rangoEdad',
    'ISC.resultadosObservacion.enfermedadesAltTiroides',
    'ISC.resultadosObservacion.enfermedadesAntecedentesCardiacos',
    'ISC.resultadosObservacion.enfermedadesCardiopatiaIsquemica',
    'ISC.resultadosObservacion.enfermedadesHepatica',
    'ISC.resultadosObservacion.enfermedadesNinguna',
    'ISC.resultadosObservacion.enfermedadesOtros',
    'ISC.resultadosObservacion.enfermedadesRenal',
    'ISC.resultadosObservacion.entorno',
    'ISC.resultadosObservacion.familiaresAlzheimer',
    'ISC.resultadosObservacion.familiaresOtraDemencia',
    'ISC.resultadosObservacion.fumador',
    'ISC.resultadosObservacion.numCigarros',
    'ISC.resultadosObservacion.numCigarrosDia_knn',
    'ISC.resultadosObservacion.numCigarrosDia_mode&rangoEdad',
    'ISC.resultadosObservacion.tratCronico',
    'ISC.resultadosObservacion.tratPsicologic',
    'ISC.resultadosObservacion.tratPsiquiatrico',
    'ISC.resultadosOtros.frecOlvidos',
    'ISC.resultadosOtros.frecUsoEmail',
    'ISC.resultadosOtros.frecUsoMovil',
    'ISC.resultadosOtros.frecUsoOrdenador',
    'ISC.resultadosOtros.frecUsoRSociales',
    'ISC.resultadosOtros.frecUsoTele',
    'ISC.resultadosOtros.nivelActFisica',
    'ISC.resultadosOtros.nivelSociabilidad',
    'ISC.resultadosReservaCognitiva.actividadLectora',
    'ISC.resultadosReservaCognitiva.cursos',
    'ISC.resultadosReservaCognitiva.escolaridadPadres',
    'ISC.resultadosReservaCognitiva.formacionMusical',
    'ISC.resultadosReservaCognitiva.juegos'
]

In [4]:
pt_columns = [column for column in df.columns if 'PT' in column ]
pz_columns = [column for column in df.columns if 'PZ' in column ]
pdc_columns = [column for column in df.columns if 'PDC' in column]
drop_columns.extend(pt_columns)
drop_columns.extend(pz_columns)
drop_columns.extend(pdc_columns)

In [5]:
df.drop(columns=drop_columns, inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)
df.head()

,ED_2Clases,ISC.FPacientes.genero,ISC.FPacientes.Edad,ISC.resultadosCribaje.Total.PD,ISC.resultadosObservacion.alcohol,ISC.resultadosObservacion.ansiedad,ISC.resultadosObservacion.depresion,ISC.resultadosObservacion.enfermedadesDiabetes,ISC.resultadosObservacion.enfermedadesHipercolesterolemia,ISC.resultadosObservacion.enfermedadesHipertension,...,TLP.Tavec.23_FP.PD,TLP.Tavec.24_Discriminabilidad.PD,TLP.Tavec.25_Sesgo.PD,TLP.Tavec.26_RI-B_frente_RI-A1.PD,TLP.Tavec.27_RL-CP_frente_RI-A5.PD,TLP.Tavec.28_RCl-Cp_frente_RCl-LP.PD,TLP.Tavec.29_RL-LP_frente_RL-CP.PD,TLP.Tavec.30_RCl-LP_frente_RL-LP.PD,TLP.Tavec.31_Recon-Ac_frente_RL-LP.PD,TLP.Tavec.32_Recon-Ac_frente_RCl-LP.PD
0,D,2,72,16,0,1,0,0,1,0,...,3,92.863551,0.483730,13.364113,5.253295,32.282205,11.618063,-8.199711,-31.422798,-14.463315
1,D,2,63,12,1,1,0,0,0,0,...,3,86.532820,-0.445374,29.354299,10.010364,0.795255,2.561297,-22.676398,-33.105191,-14.864963
2,D,2,70,10,0,1,0,0,0,0,...,2,96.943536,0.389090,86.966146,2.390008,-8.782831,4.234588,1.087507,-13.250484,-5.450347
3,D,2,79,15,0,1,0,0,0,0,...,3,87.856478,-0.422956,-39.246538,127.846164,1.784387,-14.341309,-25.921198,-54.011678,-43.942963
4,D,1,80,14,1,0,0,0,0,0,...,3,94.256142,0.378951,-13.631955,46.230693,0.802316,5.907752,-13.615024,-53.865677,-43.045912


In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((480, 68), (120, 68))

In [7]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v1'
    )

In [8]:
predictor.fit(
    train_data = df_train,
    presets = ['best_quality'],
    time_limit = 900,
    auto_stack = True,
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       9.67 GB / 15.94 GB (60.7%)
Disk Space Avail:   121.18 GB / 446.36 GB (27.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up 

(_ray_fit pid=26180) [1000]	valid_set's binary_logloss: 0.12898


(_dystack pid=30684) 	0.9283	 = Validation score   (roc_auc)
(_dystack pid=30684) 	1.63s	 = Training   runtime
(_dystack pid=30684) 	0.01s	 = Validation runtime
(_dystack pid=30684) Fitting model: LightGBM_BAG_L1 ... Training model for up to 139.51s of the 212.93s of remaining time.
(_dystack pid=30684) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.15%)
(_dystack pid=30684) 	0.9114	 = Validation score   (roc_auc)
(_dystack pid=30684) 	0.95s	 = Training   runtime
(_dystack pid=30684) 	0.03s	 = Validation runtime
(_dystack pid=30684) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 135.44s of the 208.86s of remaining time.
(_dystack pid=30684) 	0.9044	 = Validation score   (roc_auc)
(_dystack pid=30684) 	0.75s	 = Training   runtime
(_dystack pid=30684) 	0.07s	 = Validation runtime
(_dystack pid=30684) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 134.59s of the 208.0

(_ray_fit pid=29276) [1000]	valid_set's binary_logloss: 0.212424


(_dystack pid=30684) 	0.915	 = Validation score   (roc_auc)
(_dystack pid=30684) 	4.22s	 = Training   runtime
(_dystack pid=30684) 	0.01s	 = Validation runtime
(_dystack pid=30684) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 37.43s of the 110.86s of remaining time.
(_dystack pid=30684) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.03%)
(_dystack pid=30684) 	0.9411	 = Validation score   (roc_auc)
(_dystack pid=30684) 	6.16s	 = Training   runtime
(_dystack pid=30684) 	0.09s	 = Validation runtime
(_dystack pid=30684) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 22.14s of the 95.56s of remaining time.
(_dystack pid=30684) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.70%)
(_dystack pid=30684) 	0.9218	 = Validation score   (roc_auc)
(_dystack pid=30684) 	17.86s	 = Training   runtime
(

In [9]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           WeightedEnsemble_L3   0.946559     roc_auc       3.931955  420.437253                0.002006           0.129760            3       True         62
1           WeightedEnsemble_L2   0.945899     roc_auc       0.885947  122.070367                0.000000           0.174198            2       True         44
2           LightGBM_r96_BAG_L2   0.937530     roc_auc       3.288711  338.574011                0.033755           3.262652            2       True         61
3               CatBoost_BAG_L2   0.937253     roc_auc       3.306703  358.258234                0.051747          22.946875            2       True         49
4            CatBoost_r9_BAG_L2   0.935829     roc_auc       3.321305  386.421552                0.066349          51.110193            2 

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [10]:
auxiliary_metrics = ['balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [11]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9410456062291435
Evaluations on test data:
{
    "roc_auc": 0.9410456062291435,
    "accuracy": 0.8916666666666667,
    "balanced_accuracy": 0.89293659621802,
    "mcc": 0.7864201151777477,
    "f1": 0.8925619834710744,
    "precision": 0.8571428571428571,
    "recall": 0.9310344827586207
}


{'roc_auc': 0.9410456062291435,
 'accuracy': 0.8916666666666667,
 'balanced_accuracy': 0.89293659621802,
 'mcc': 0.7864201151777477,
 'f1': 0.8925619834710744,
 'precision': 0.8571428571428571,
 'recall': 0.9310344827586207}

In [12]:
predictor.leaderboard(
    df_train,
    extra_metrics = auxiliary_metrics,
    silent=True
)

,model,score_test,balanced_accuracy,f1,f1_macro,f1_micro,roc_auc,average_precision,precision,recall,...,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.012748,0.004867,0.007873,0.012748,0.004867,0.007873,1,True,2
1,LightGBM_BAG_L1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.015811,0.016315,1.732943,0.015811,0.016315,1.732943,1,True,4
2,CatBoost_r177_BAG_L1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.031321,0.078985,7.096169,0.031321,0.078985,7.096169,1,True,14
3,CatBoost_BAG_L1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.031831,0.053791,7.883599,0.031831,0.053791,7.883599,1,True,7
4,LightGBMLarge_BAG_L1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.037572,0.027023,3.419167,0.037572,0.027023,3.419167,1,True,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,NeuralNetFastAI_r156_BAG_L1,0.984495,0.937218,0.940000,0.937391,0.937500,0.984495,0.984801,0.910853,0.971074,...,roc_auc,0.354079,0.109216,8.455306,0.354079,0.109216,8.455306,1,True,43
58,NeuralNetFastAI_r143_BAG_L1,0.973957,0.908188,0.910569,0.908276,0.908333,0.973957,0.973814,0.896000,0.925620,...,roc_auc,0.381469,0.105157,10.967052,0.381469,0.105157,10.967052,1,True,41
59,NeuralNetFastAI_r103_BAG_L1,0.956247,0.885461,0.885655,0.885416,0.885417,0.956247,0.952486,0.891213,0.880165,...,roc_auc,0.125608,0.087191,6.235137,0.125608,0.087191,6.235137,1,True,38
60,NeuralNetFastAI_r11_BAG_L1,0.942583,0.866588,0.868852,0.866630,0.866667,0.942583,0.937573,0.861789,0.876033,...,roc_auc,0.177380,0.060544,7.208525,0.177380,0.060544,7.208525,1,True,34


In [ ]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['TLP.MMSE.Fijacion.PD']
Computing feature importance via permutation shuffling for 66 features using 480 rows with 5 shuffle sets...
	1832.24s	= Expected runtime (366.45s per shuffle set)
2025-01-20 21:58:05,095	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-01-20 21:58:05,778	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-01-20 21:58:05,782	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
